In [ ]:
from materials_2050_api_client import API_Client
import os

### Setup developer token

In [ ]:
# Set the environment variable
os.environ['DEVELOPER_TOKEN']= ''

# Now you can access it using getenv
developer_token = os.getenv('DEVELOPER_TOKEN')

### Create Client

In [ ]:
client = API_Client(base_api_url = "https://app.2050-materials.com/", developer_token = developer_token)

### Get products

In [ ]:
# API
# products = client.get_products()

# Open API
# products = client.get_products_open_api()

### Filters

In [ ]:
# Fetch all filters
# all_filters = client.get_filters()

# Fetch filters for open API
# open_filters = client.get_open_filters()

# Get product types dictionary
product_types = client.get_product_types()

# Get material types dictionary
material_types = client.get_material_types()

### Apply Filters to open API

In [ ]:
filters = {
#     'name': 'weber MTL Wet room render (weber MTL Märkätilalaasti)',
    'product_type': product_types['Building boards (Fire)'],
    'material_types': material_types['Gypsum'],
#     'country': 'France',
#     'manufacturing_continent': 'Europe',
#     'product_url': [
#         "https://app.2050-materials.com/product/details_designer/dalsan-alci-sanayi-ve-ticaret-a-s-rightfinish-skimcoat-thin-coat-plaster",
#         "https://app.2050-materials.com/product/details_designer/benjamin-moore-aura-exterior-paint-flat-flat-n629",
#         "https://app.2050-materials.com/product/details_designer/saint-gobain-finland-oy-weber-mtl-wet-room-render-weber-mtl-markatilalaasti/"
#     ]
}

filtered_products = client.get_filtered_data_open_api(**filters)
filtered_products